# 数据预处理

## q1

### BPE

> start: learn from [MartinLwx's blog | https://martinlwx.github.io/zh-cn/the-bpe-tokenizer/]

#### 训练流程

假设有文档集$D = d_{1},d_{2},...$

- 先将文档$d$变成单词列表,比如使用简单空格分词
- 统计每个单词 $w$ 在所有文档 $D$ 中的出现频率，并计算初始`字符集 alphabet`(所有文档 $D$ 中不同的字符集合) 作为一开始的 `Vocab`(包括后面的`</w>`)
- 先将每个单词划分为一个个 utf-8 char，称为一个划分，比如 `highest -> h, i, g, h, e, s, t`
- 在每个单词的划分最后面加上 `</w>`，那么现在 `highest -> h, i, g, h, e, s, t, </w>`
- 重复下面步骤直到满足两个条件中的任意一个：1）Vocab 达到上限。2）达到最大迭代次数 
  - 找到**最经常一起出现的 pair**，并记录这个合并规则，放在 `merge table(存储合并规则的字典或哈希表)` 里面，同时把合并之后的结果放到 Vocab 里面
  - 更新所有单词的划分，假设我们发现 `(h, i)` 最经常一起出现，那么 `hi` 就会被添加到 Vocab 里面，同时修改划分方式为：`highest -> hi, g, h, e, s, t, </w>`
> 统计词频: 简单化找最常出现pair  

> 为什么要加`</w>`: 还原输入单词完整性,标记单词边界

#### 应用流程

假设要处理文本`s`

- 把`s`拆分成单词列表 ,每个单词拆分成uft-8 char, 单词末尾加`</w>`
- 遍历 merge table，并检查每个合并规则是否可以用来更新每个单词的划分，可以的话就合并更新

#### 例子

In [ ]:
# 定义语料库
corpus = ["highest", "higher", "lower", "lowest", "cooler", "coolest"]

# 统计词频
count = 1

# 拆分词汇并加</w>
{
    "highest": ["h", "i", "g", "h", "e", "s", "t", "</w>"],
    "higher": ["h", "i", "g", "h", "e", "r", "</w>"],
    "lower": ["l", "o", "w", "e", "r", "</w>"],
    "lowest": ["l", "o", "w", "e", "s", "t", "</w>"],
    "cooler": ["c", "o", "o", "l", "e", "r", "</w>"],
    "coolest": ["c", "o", "o", "l", "e", "s", "t", "</w>"],
}

#合并高频pair "es"
{
    "highest": ["h", "i", "g", "h", "es", "t", "</w>"],
    "higher": ["h", "i", "g", "h", "e", "r", "</w>"],
    "lower": ["l", "o", "w", "e", "r", "</w>"],
    "lowest": ["l", "o", "w", "es", "t", "</w>"],
    "cooler": ["c", "o", "o", "l", "e", "r", "</w>"],
    "coolest": ["c", "o", "o", "l", "es", "t", "</w>"],
}

# 合并高频pair "est"
{
    "highest": ["h", "i", "g", "h", "est", "</w>"],
    "higher": ["h", "i", "g", "h", "e", "r", "</w>"],
    "lower": ["l", "o", "w", "e", "r", "</w>"],
    "lowest": ["l", "o", "w", "est", "</w>"],
    "cooler": ["c", "o", "o", "l", "e", "r", "</w>"],
    "coolest": ["c", "o", "o", "l", "est", "</w>"],
}

# ...

#### BPE的Huggingface实现

In [ ]:
from tokenizers import CharBPETokenizer

# Instantiate tokenizer
tokenizer = CharBPETokenizer()

tokenizer.train_from_iterator(
    corpus, # 语料库
    vocab_size=17, # 词汇表大小
    min_frequency=2, # 最小词频
)

#### 手动实现BPE

In [ ]:
from collections import defaultdict, Counter
from pprint import pprint


class BPE:
    def __init__(
        self,
        corpus: list[str], # 语料库
        vocab_size: int, # 词汇表大小
        max_iter: int | None = None, # 最大迭代次数
        debug: bool = False,
    ):
        self.corpus = corpus
        self.vocab_size = vocab_size
        self.vocab = []
        self.word_freq = Counter() # 词频
        self.splits = {}  # e.g. highest: [high, est</w>] # 拆分词汇
        self.merges = {}  # e.g. [high, est</w>]: highest # 合并词汇
        self.max_iter = max_iter
        self.debug = debug

    def train(self):
        """Train a BPE Tokenizer"""
        # count the word frequency
        for document in self.corpus:
            # split each document in corpus by whitespace
            words = document.split()
            self.word_freq += Counter(words)
            # Counter 返回一个字典，字典的键是单词，值是单词出现的个数

        # initialize the self.splits
        for word in self.word_freq:
            self.splits[word] = list(word) + ["</w>"]
        # 拆分词汇并加</w>

        if self.debug:
            print(f"Init splits: {self.splits}")

        alphabet = set() # 字符集合
        for word in self.word_freq:
            alphabet |= set(list(word)) # 求字符集合的并集
        alphabet.add("</w>") # 添加结束符

        self.vocab = list(alphabet)
        self.vocab.sort() # 可以不排序

        cnt = 0
        while len(self.vocab) < self.vocab_size:
            if self.max_iter and cnt >= self.max_iter: # 达到最大迭代次数
                break

            # find the most frequent pair
            pair_freq = self.get_pairs_freq()

            if len(pair_freq) == 0:
                print("No pair available")
                break

            pair = max(pair_freq, key=pair_freq.get) # 找到最高频的pair, pair其实是排序后的字符组合

            self.update_splits(pair[0], pair[1]) # 更新拆分词汇

            if self.debug:
                print(f"Updated splits: {self.splits}")

            self.merges[pair] = pair[0] + pair[1]

            self.vocab.append(pair[0] + pair[1])

            if self.debug:
                print(
                    f"Most frequent pair({max(pair_freq.values())} times) "
                    f"is : {pair[0]}, {pair[1]}. Vocab size: {len(self.vocab)}"
                )

            cnt += 1
    
    def update_splits(self, lhs: str, rhs: str):
        """If we see lhs and rhs appear consecutively, we merge them"""
        for word, word_split in self.splits.items():
            new_split = []
            cursor = 0
            while cursor < len(word_split): # 遍历拆分词汇
                if (
                    word_split[cursor] == lhs # 如果当前字符是lhs
                    and cursor + 1 < len(word_split) # 并且后面还有字符
                    and word_split[cursor + 1] == rhs # 并且后面的字符是rhs
                ):
                    new_split.append(lhs + rhs) # 合并
                    cursor += 2 # 跳过lhs和rhs
                else:
                    new_split.append(word_split[cursor]) # 不合并
                    cursor += 1 # 继续遍历
            self.splits[word] = new_split # 更新拆分词汇

            # if word_split != new_split:
            #     print(f"old: {word_split}")
            #     print(f"new: {new_split}")

    def get_pairs_freq(self) -> dict:
        """Compute the pair frequency"""
        pairs_freq = defaultdict(int) # pair频率, 默认值为0
        for word, freq in self.word_freq.items():
            split = self.splits[word] # 拆分词汇
            for i in range(len(split)):
                if i + 1 < len(split):
                    pairs_freq[(split[i], split[i + 1])] += freq # 计算pair频率, freq是词频

        return pairs_freq

    def tokenize(self, s: str) -> list[str]:
        splits = [list(t) + ["</w>"] for t in s.split()] # 拆分字符串并加</w>

        for lhs, rhs in self.merges:
            for idx, split in enumerate(splits): # 遍历拆分词汇, idx是索引, split是拆分词汇, enumerate返回索引和元素
                new_split = []
                cursor = 0
                while cursor < len(split):
                    if (
                        cursor + 1 < len(split)
                        and split[cursor] == lhs
                        and split[cursor + 1] == rhs
                    ):
                        new_split.append(lhs + rhs)
                        cursor += 2
                    else:
                        new_split.append(split[cursor])
                        cursor += 1
                assert "".join(new_split) == "".join(split)
                splits[idx] = new_split

        return sum(splits, []) # 合并拆分词汇, sum函数的第二个参数是[]，表示从空列表开始累加

corpus = ["highest", "higher", "lower", "lowest", "cooler", "coolest"]
bpe = BPE(corpus, 17, debug=False)
bpe.train()
bpe.tokenize("".join(corpus))

#### 局限

> 把文档变成一个个单词我们这里用的是空格划分，但是像中文的话，空格并不是单词之间的边界

### WorkPiece

WorkPiece是BPE的一种变种, 本质上二者的训练过程相差不多, 区别在于:  
- BPE根据pair的频率决定合并策略
- WorkPiece是选择使得语言模型点互信息最大的相邻pair加入词表  

设待合并两字词为$x$, $y$, 合并后字词为$z$, 点互信息$I(z)$有:
$$ I(z) = I(x, y) = log\frac{P(x, y)}{P(x)P(y)} = log\frac{P(z)}{P(x)P(y)} $$

#### 与BPE的不同
> 在大量文本训练时, BPE的合并策略, 使得其在编码时更倾向于把单词按"时态"合并  
> 而WorkPiece更倾向于按概率合并, 单词内部很可能有一个或多个有"意义"的单词

### Unigram

Unigram与前两种分词模型最大区别是, **先初始一个大词表，接着通过语言模型评估不断减少词表**，直到限定词汇量

#### 训练过程

> 假设: 所有subword的出现都是独立的，并且subword序列由subword出现概率的乘积产生, 就是语言模型的链式概率,即:$\vec{x} = (x_{1},...,x_{M})$, $P(x) = \prod_{i=1}^{M}p(x_{i})$
  
优化目标:  
$$ \Lambda = \sum_{s=1}^{\vert{D}\vert}log(P(X^{(s)})) = \sum_{s=1}^{\vert{D}\vert}log(\sum P(x)) $$

- 初始时构建一个相当大的字符集
- 重复以下步骤，直到字典尺寸`|V|`减小到期望值
  - 固定词典，通过EM算法优化$p(x)$
  - 计算每一个子词$loss_{i}$，loss代表如果将某个词去掉，上述优化函数值会减少多少
  - 根据$loss$排序，保留$loss$最高的n%个子词（因为要使得似然概率最大，则根据$loss$定义，应该保留$loss$大的）同时保留所有的单字符，从而避免$OOV$情况

## P2

### sentencepiece训练

In [11]:

import sentencepiece as spm

spm.SentencePieceTrainer.train('--input=train/西游记.txt --model_prefix=train/sentencepiece/ans --vocab_size=20000 --model_type=bpe')

### jieba训练

In [3]:
import jieba
import pandas as pd

# 读取.txt数据
with open('train/西游记.txt', 'r', encoding='utf-8') as f:
    data = f.read()

# 分词
data_acc = jieba.cut(data, cut_all=False, HMM=True) # 精确模式分词

# 保存分词结果
df = pd.DataFrame(data_acc, columns=['word'])
df.to_csv('train/jieba/西游记_cut.txt', index=False)
